# Classificació

### Libraries and loading data

In [19]:
import pandas as pd
from sklearn.svm import SVC
from skopt import BayesSearchCV
import numpy as np
np.int = int

from IPython.display import display, HTML
show_html = lambda html: display(HTML(html))

from sklearn.metrics import  ConfusionMatrixDisplay, classification_report

import warnings
warnings.filterwarnings("ignore")

In [1]:
df_train = pd.read_csv('data_train_class.csv')
df_test = pd.read_csv('data_test_class.csv')

In [2]:
df_train.head()

,BPM,Energy,Danceability,Loudness,Liveness,Valence,Length,Acousticness,Speechiness,Popularity,...,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9
0,136,78,45,-4,67,24,187,11,6,73,...,0,0,1,0,0,0,0,0,0,0
1,134,32,28,-17,11,14,1412,37,4,58,...,1,0,0,0,0,0,0,0,0,0
2,93,93,44,-3,8,40,203,1,5,62,...,0,0,0,0,0,1,0,0,0,0
3,93,48,74,-9,11,36,285,3,4,73,...,0,0,0,0,0,0,0,0,1,0
4,117,75,61,-9,39,97,394,1,3,49,...,0,0,0,0,1,0,0,0,0,0


In [3]:
x_train = df_train.drop(columns=['is_recent'])
y_train = df_train['is_recent']

x_test = df_test.drop(columns=['is_recent'])
y_test = df_test['is_recent']

clss = ['Pre-2000', 'Post-2000']

In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train))
x_test = pd.DataFrame(scaler.transform(x_test))

x_train.describe().T

,count,mean,std,min,25%,50%,75%,max
0,1395.0,0.501775,0.166176,0.0,0.375000,0.494048,0.589286,1.0
1,1395.0,0.578314,0.231223,0.0,0.400000,0.600000,0.768421,1.0
2,1395.0,0.504476,0.176126,0.0,0.383721,0.511628,0.627907,1.0
3,1395.0,0.720401,0.144714,0.0,0.640000,0.760000,0.840000,1.0
4,1395.0,0.179839,0.177908,0.0,0.072917,0.104167,0.223958,1.0
5,1395.0,0.489277,0.257525,0.0,0.270833,0.468750,0.697917,1.0
6,1395.0,0.123729,0.076020,0.0,0.083969,0.109924,0.143511,1.0
7,1395.0,0.292758,0.292017,0.0,0.040816,0.183673,0.500000,1.0
8,1395.0,0.056861,0.084317,0.0,0.018868,0.037736,0.056604,1.0
9,1395.0,0.560367,0.164901,0.0,0.448276,0.586207,0.689655,1.0


## Models lineals

### SVM lineal

In [20]:
param = {'C':10**np.linspace(-3,3,101)}

lsvc =  SVC(kernel='linear', max_iter=25000, random_state=0, class_weight = "balanced") # The balanced version doesn't work very well
lsvc_gs = BayesSearchCV(lsvc,param,n_iter=15, cv=10, n_jobs=-1, refit=True, random_state=0)
lsvc_gs.fit(x_train, y_train);

/usr/local/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=25000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=25000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=25000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=25000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=25000).  Con

In [14]:
show_html(pd.DataFrame(lsvc_gs.cv_results_).loc[:,['params', 'mean_test_score','rank_test_score']].sort_values(by='rank_test_score').head().to_html())

,params,mean_test_score,rank_test_score
12,{'C': 54.954087385762485},0.759152,1
5,{'C': 109.64781961431851},0.758433,2
13,{'C': 72.44359600749891},0.758433,2
14,{'C': 15.848931924611142},0.757718,4
7,{'C': 1.1481536214968828},0.754866,5


In [17]:
print(classification_report(lsvc_gs.predict(x_test), y_test, target_names=clss))

              precision    recall  f1-score   support

    Pre-2000       0.71      0.83      0.77       301
   Post-2000       0.79      0.66      0.72       298

    accuracy                           0.75       599
   macro avg       0.75      0.75      0.74       599
weighted avg       0.75      0.75      0.74       599

